In [1]:
import os

In [2]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'h:\\Projects\\mlops\\first\\pizza-steak-end-to-end'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from PizzaSteakClassifier.constants import *
from PizzaSteakClassifier.utils.common import read_yaml_file, create_directories

In [8]:
config = CONFIG_FILE_PATH
print(config)
yaml = read_yaml_file(config)
print(yaml)

config\config.yaml
[2023-12-30 16:31:26,278 - INFO - common - YAML file loaded successfully: config\config.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://storage.googleapis.com/heispv/pizza-steak-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml_file(config_file_path)
        self.params = read_yaml_file(params_file_path)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import urllib.request as request
import zipfile
from PizzaSteakClassifier import logger
from PizzaSteakClassifier.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        """
        Downloads the data from the specified source URL and saves it locally.

        If the local data file already exists, it logs a message indicating its size.
        If the local data file does not exist, it downloads the file and logs the download details.

        Returns:
            None
        """
        if not os.path.exists(self.config.local_data_file):
            file_name, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"Downloaded {file_name} of size {get_size(Path(self.config.local_data_file))}")
        else:
            logger.info(f"Data file {self.config.local_data_file} already exists with the size {get_size(Path(self.config.local_data_file))}")

    def unzip_data(self):
            """
            Unzips the data file to the specified directory.

            Args:
                self.config.local_data_file (str): The path to the local data file.
                self.config.unzip_dir (str): The directory where the data file will be extracted.

            Returns:
                None
            """
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(self.config.unzip_dir)


In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise(e)

[2023-12-30 16:31:28,015 - INFO - common - YAML file loaded successfully: config\config.yaml]
[2023-12-30 16:31:28,017 - INFO - common - YAML file loaded successfully: params.yaml]
[2023-12-30 16:31:28,018 - INFO - common - Directory created: artifacts]
[2023-12-30 16:31:28,112 - INFO - common - Directory created: artifacts/data_ingestion]
[2023-12-30 16:31:56,389 - INFO - 3773734432 - Downloaded artifacts/data_ingestion/data.zip of size 106860.9385 KB
X-GUploader-UploadID: ABPtcPrXwtWRttUP58VHLbqdnfBhf5bgytZ3MNS0wf8zKrD8umZ7ZxuHpMjTp1keJWpPRT2lnvk
Expires: Sat, 30 Dec 2023 16:31:28 GMT
Date: Sat, 30 Dec 2023 15:31:28 GMT
Cache-Control: public, max-age=3600
Last-Modified: Sat, 30 Dec 2023 15:26:07 GMT
ETag: "d38a02f09adc8c4b5db61ab4e1b18da0"
x-goog-generation: 1703949967397828
x-goog-metageneration: 2
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 109425601
Content-Type: application/x-zip-compressed
x-goog-hash: crc32c=NOJixA==
x-goog-hash: md5=04oC8JrcjEtdthq04